In [73]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("..")

from mapper.geometry import from_twist, to_twist

In [10]:
pose_graph = pickle.load(open("pose_graph.pkl", "rb"))
map_points = pickle.load(open("map_points.pkl", "rb"))

In [11]:
[from_twist(pose_graph.nodes[n]["pose"]) for n in pose_graph]

[(array([[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]),
  array([[0.],
         [0.],
         [0.]])),
 (array([[ 0.99987317,  0.012289  , -0.01013085],
         [-0.01142025,  0.99658692,  0.08175627],
         [ 0.01110098, -0.0816302 ,  0.99660086]]),
  array([[ 0.1752848 ],
         [-0.62653719],
         [-0.75942504]])),
 (array([[ 0.99957479,  0.02630918, -0.01257261],
         [-0.02366484,  0.98386214,  0.17735634],
         [ 0.01703582, -0.1769834 ,  0.98406639]]),
  array([[ 0.33454752],
         [-1.41361467],
         [-1.87494293]]))]

In [50]:
poses = [
    (np.array([[1., 0., 0.],
               [0., 1., 0.],
               [0., 0., 1.]]
             ), 
     np.array([[0.],[0.],[0.]])),
    (np.array([[1., 0., 0.],
               [0., 0.8660254, -0.5000000],
               [0., 0.5000000,  0.8660254]]), 
     np.array([[1.],[0.],[0.]])
    ),
    (np.array([[1., 0., 0.],
               [0., 0.5000000, -0.8660254],
               [0., 0.8660254,  0.5000000]]), 
     np.array([[2.],[0.],[0.]])
    )
]

### Estimate rotational velocity

In [68]:
velocity_R = np.matmul(poses[-2][0].T, poses[-1][0])
velocity_R

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.8660254 , -0.49999999],
       [ 0.        ,  0.49999999,  0.8660254 ]])

In [70]:
next_R = np.matmul(velocity_R, poses[2][0])
next_R

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  5.67665803e-09, -9.99999990e-01],
       [ 0.00000000e+00,  9.99999990e-01,  5.67665803e-09]])

### Estimate translational velocity

In [ ]:
velocity_t = poses[-1][1] - poses[-2][1]

In [37]:
# predict next pose
next_t = velocity + poses[-1][1]

In [33]:
next_t

array([[3.],
       [0.],
       [0.]])

In [58]:
poses[-1][0]

array([[ 1.       ,  0.       ,  0.       ],
       [ 0.       ,  0.5      , -0.8660254],
       [ 0.       ,  0.8660254,  0.5      ]])

### Wrap in a function

In [71]:
def estimate_velocity(prev_pose, current_pose):
    """Returns translational and rotational velocity 
    that transform prev_pose into current_pose."""
    R0, t0 = from_twist(prev_pose)
    R1, t1 = from_twist(current_pose)
    # estimate translational velocity
    vt = t1 - t0
    # estimate rotational velocity
    vR = np.matmul(R0.T, R1)
    velocity = (vR, vt)
    return velocity


def predict_next_pose(pose, velocity):
    """Predicts next pose from pose with constant velocity model."""
    R, t = from_twist(pose)
    vR, vt = velocity
    next_R = np.matmul(vR, R)
    next_t = vt + t
    return to_twist(next_R, next_t)

In [75]:
prev_pose = to_twist(poses[0][0], poses[0][1])
current_pose = to_twist(poses[1][0], poses[1][1])

velocity = estimate_velocity(prev_pose, current_pose)

next_pose = predict_next_pose(current_pose, velocity)

In [77]:
from_twist(next_pose)

(array([[ 1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.5       , -0.86602541],
        [ 0.        ,  0.86602541,  0.5       ]]),
 array([[2.],
        [0.],
        [0.]]))